In [2]:
#!pip install keras

In [15]:
import os
import matplotlib.pyplot as plt
 #to read image
from skimage.io import imread
#to resize image to uniform sizes
from skimage.transform import resize

import tensorflow as tf
import numpy as np
import pandas as pd

import keras
from keras.layers import Dense, Flatten
from keras.models import Model
from keras.preprocessing.image import ImageDataGenerator #, load_img, img_to_array #
from tensorflow.keras.utils import load_img, img_to_array

In [16]:
tf.keras.applications.inception_v3.preprocess_input
from keras.applications.inception_v3 import preprocess_input

In [9]:
base_model = tf.keras.applications.inception_v3.InceptionV3(input_shape=(256,256,3),
                                                           include_top=False
                                                           )


87910968/87910968 [==============================] - 3s 0us/step


In [6]:
for layer in base_model.layers:
    layer.trainable=False

In [7]:
#read on activation functions
X = Flatten()(base_model.output)
X =Dense(units=3, activation = 'softmax')(X)

In [8]:
model = Model(base_model.input, X)

#compile model
model.compile(optimizer='adam',loss= keras.losses.binary_crossentropy, metrics=['accuracy'])

#summary
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 256, 256, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 127, 127, 32)         864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 127, 127, 32)         96        ['conv2d[0][0]']              
 Normalization)                                                                                   
                                                                                                  
 activation (Activation)     (None, 127, 127, 32)         0         ['batch_normalization[0][0

In [10]:
datagen = ImageDataGenerator(featurewise_center = True, rotation_range=0.4,
                                  width_shift_range=0.3,
                                  horizontal_flip=True,
                                  preprocessing_function= preprocess_input,
                                  zoom_range=0.4,
                                  shear_range = 0.4)



In [10]:
train_data = datagen.flow_from_directory(directory='/content/drive/MyDrive/Brain_tumor_classification/Brain_tumor_Data/train',
                                              target_size=(256,256),
                                              batch_size=32)

Found 1845 images belonging to 3 classes.


In [11]:
train_data.class_indices

{'glioma': 0, 'meningioma': 1, 'pituitary': 2}

In [11]:
test_data = datagen.flow_from_directory(directory='/content/drive/MyDrive/Brain_tumor_classification/Brain_tumor_Data/test',
                                              target_size=(256,256),
                                              batch_size=32)

Found 620 images belonging to 3 classes.


In [13]:
val_data = datagen.flow_from_directory(directory='/content/drive/MyDrive/Brain_tumor_classification/Brain_tumor_Data/val',
                                              target_size=(256,256),
                                              batch_size=32)

Found 612 images belonging to 3 classes.


In [16]:
'''from keras.callbacks import ModelCheckpoint, EarlyStopping

mc = ModelCheckpoint(filepath='/content/drive/MyDrive/Brain_tumor_classification/Brain_tumor_Data/best_model.h5',
                  monitor = 'accuracy',
                  verbose=1,
                  save_best_only=True)
es= EarlyStopping(monitor='accuracy',
                 min_delta=0.01,
                 patience=10,
                 verbose=1)

cb=[mc,es]'''

In [17]:
'''his = model.fit_generator(train_data,
                          steps_per_epoch=56,
                          epochs=30,
                          validation_data=val_data,
                          callbacks=cb)'''


<ipython-input-17-83f3d0f1e027>:1: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  his = model.fit_generator(train_data,


Epoch 1/30
56/56 [==============================] - ETA: 0s - loss: 0.9673 - accuracy: 0.6996
Epoch 1: accuracy improved from -inf to 0.69961, saving model to /content/drive/MyDrive/Brain_tumor_classification/Brain_tumor_Data/best_model.h5


/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3079: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


56/56 [==============================] - 434s 8s/step - loss: 0.9673 - accuracy: 0.6996 - val_loss: 0.4806 - val_accuracy: 0.7794
Epoch 2/30
56/56 [==============================] - ETA: 0s - loss: 0.6224 - accuracy: 0.7631
Epoch 2: accuracy improved from 0.69961 to 0.76305, saving model to /content/drive/MyDrive/Brain_tumor_classification/Brain_tumor_Data/best_model.h5
56/56 [==============================] - 79s 1s/step - loss: 0.6224 - accuracy: 0.7631 - val_loss: 0.5647 - val_accuracy: 0.7500
Epoch 3/30
56/56 [==============================] - ETA: 0s - loss: 0.5814 - accuracy: 0.7715
Epoch 3: accuracy improved from 0.76305 to 0.77148, saving model to /content/drive/MyDrive/Brain_tumor_classification/Brain_tumor_Data/best_model.h5
56/56 [==============================] - 76s 1s/step - loss: 0.5814 - accuracy: 0.7715 - val_loss: 0.6599 - val_accuracy: 0.7533
Epoch 4/30
56/56 [==============================] - ETA: 0s - loss: 0.6117 - accuracy: 0.7866
Epoch 4: accuracy improved from 

KeyboardInterrupt: ignored

In [1]:
from keras.models import load_model

In [2]:
model = load_model('best_model.h5')

In [4]:
# Assuming you have your model and test data directory ready
from tensorflow.keras.preprocessing.image import ImageDataGenerator

test_datagen = ImageDataGenerator(rescale=1./255)  # Normalize pixel values
test_data = test_datagen.flow_from_directory(
    'test',  # Replace with your test data directory
    target_size=(256, 256),  # Resize images to (224, 224)
    batch_size=32,  # Set batch size
    class_mode='categorical',  # Set class mode to 'categorical' for multi-class classification
    shuffle=False  # Don't shuffle the data
)

# Evaluate model on test data
test_loss, test_accuracy = model.evaluate(test_data)

print("Test Loss:", test_loss)
print("Test Accuracy:", test_accuracy)


Found 615 images belonging to 3 classes.
 1/20 [>.............................] - ETA: 23:38 - loss: 1.1781 - accuracy: 0.9062


KeyboardInterrupt



In [5]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn.metrics import confusion_matrix, roc_curve, auc


# Predict probabilities for test data
y_pred_prob = model.predict(test_data)

# Get predicted labels
y_pred = np.argmax(y_pred_prob, axis=1)

# Get true labels
y_true = test_data.classes

# Generate confusion matrix
cm = confusion_matrix(y_true, y_pred)

# Plot confusion matrix
plt.imshow(cm, cmap=plt.cm.Blues)
plt.title('Confusion Matrix')
plt.colorbar()
tick_marks = np.arange(3)
plt.xticks(tick_marks, ['glioma', 'meningioma', 'pituitary'])
plt.yticks(tick_marks, ['glioma', 'meningioma', 'pituitary'])
plt.xlabel('Predicted Labels')
plt.ylabel('True Labels')
plt.show()

# Plot ROC curve
fpr = dict()
tpr = dict()
roc_auc = dict()

for i in range(3):
    fpr[i], tpr[i], _ = roc_curve(y_true, y_pred_prob[:, i], pos_label=i)
    roc_auc[i] = auc(fpr[i], tpr[i])

plt.figure()
for i in range(3):
    plt.plot(fpr[i], tpr[i], label='{0} (AUC = {1:.2f})'.format(['glioma', 'meningioma', 'pituitary'][i], roc_auc[i]))

plt.plot([0, 1], [0, 1], 'k--')
plt.xlim(0.0, 1.0)
plt.ylim(0.0, 1.05)
plt.xlabel('False Positive Rate')
plt.ylabel('True Positive Rate')
plt.title('ROC Curve')
plt.legend(loc='lower right')
plt.show()

 3/20 [===>..........................] - ETA: 17:03

KeyboardInterrupt: 

In [24]:
print(cm)

[[245  41   0]
 [  8 132   3]
 [ 12  79 100]]


In [6]:
%%writefile brain_tumor.py

import streamlit as st
import pandas as pd
import numpy as np
import tensorflow as tf
from PIL import Image
from io import BytesIO
from tensorflow.keras.utils import load_img, img_to_array
#from keras.preprocessing.image import img_to_array
from keras.applications.inception_v3 import preprocess_input
from keras.models import load_model

model = load_model('best_model.h5')

classes = {0:'glioma',1:'meningioma', 2:'pituitary'}

img_file = st.file_uploader('select an image', type=['jpg','png','jpeg','gif','jfif','heic'])

if img_file is not None :
    img = Image.open(img_file)
    st.image(img,caption='Upload image succesfully')
    
    if st.button('predict'):
        img = img.resize((256,256))
        i = img_to_array(img)
        i = preprocess_input(i)
        input_arr = np.array([i])
        
        y_out = np.argmax(model.predict(input_arr))
        y_out1 = classes[y_out]
        
        #if y_out1 = 0:
        st.write(f'This image is a {y_out1}')
        #else:
            #st.write(f'This image is a {y_out1}')

Writing brain_tumor.py
